In [1]:
datadir="/home/qjh/anomaly_data/datasets/mvtec"
texture_source_dir="/home/qjh/anomaly_data/datasets/dtd/images"
savedir = "./saved_model"
target = "bottle"
fg_root_path = "/home/qjh/anomaly_data/result/foreground"
fg_path = fg_root_path + f"/{target}/train/good"
use_mask = True

In [2]:
from omegaconf import OmegaConf
import torch
from main import run

config_dict = {
    "EXP_NAME": "MemSeg",
    "SEED": 42,
    "DATASET": {
        "datadir": datadir,
        "texture_source_dir": texture_source_dir,
        "anomaly_mask_info": "./anomaly_mask.json",
        "target": target,
        "resize": [288, 288],
        "imagesize": 256,
        "structure_grid_size": 8,
        "transparency_range": [0.15, 1.0],
        "perlin_scale": 6,
        "min_perlin_scale": 0,
        "perlin_noise_threshold": 0.5,
        "fg_path": fg_path,
        "use_mask": use_mask,
        "bg_threshold" : 30,
        "bg_reverse"   : True
    },
    
    "DATALOADER": {
        "batch_size": 8,
        "num_workers": 0
    },
    
    "MEMORYBANK": {
        "nb_memory_sample": 30
    },
    
    "MODEL": {
        "feature_extractor_name": "resnet18"
    },
    
    "TRAIN": {
        "batch_size": 6,
        "num_training_steps": 5,
        "l1_weight": 0.6,
        "focal_weight": 0.4,
        "focal_alpha": None,
        "focal_gamma": 4,
        "use_mlflow": True
    },
    
    "OPTIMIZER": {
        "lr": 0.003,
        "weight_decay": 0.0005
    },
    
    "SCHEDULER": {
        "min_lr": 0.0001,
        "warmup_ratio": 0.1,
        "use_scheduler": True
    },
    
    "LOG": {
        "log_interval": 1,
        "eval_interval": 10
    },
    
    "RESULT": {
        "savedir": savedir
    }
}

cfg = OmegaConf.create(config_dict)

In [3]:
assert cfg.DATASET.target is not None, "Please set DATASET.target"

run(cfg)

torch.cuda.empty_cache()

Device: cuda:0
Loading pretrained weights from url (https://download.pytorch.org/models/resnet18-5c106cde.pth)
Update 30 normal samples in memory bank
TRAIN [   1/5] Loss: 0.4290 (0.4290) L1 Loss: 0.5998 (0.5998) Focal Loss: 0.1726 (0.1726) LR: 1.000e-04 Time: 1.660s,    4.82/s (1.660s,    4.82/s) Data: 0.694 (0.694)
TRAIN [   2/5] Loss: 0.4077 (0.4183) L1 Loss: 0.5847 (0.5923) Focal Loss: 0.1422 (0.1574) LR: 2.723e-03 Time: 0.830s,    9.64/s (1.245s,    6.43/s) Data: 0.632 (0.663)
TRAIN [   3/5] Loss: 0.2408 (0.3592) L1 Loss: 0.3882 (0.5242) Focal Loss: 0.0198 (0.1115) LR: 1.998e-03 Time: 0.761s,   10.51/s (1.084s,    7.38/s) Data: 0.567 (0.631)
TRAIN [   4/5] Loss: 0.1444 (0.3055) L1 Loss: 0.2376 (0.4526) Focal Loss: 0.0045 (0.0848) LR: 1.102e-03 Time: 0.797s,   10.03/s (1.012s,    7.90/s) Data: 0.605 (0.624)
TRAIN [   5/5] Loss: 0.1054 (0.2654) L1 Loss: 0.1749 (0.3971) Focal Loss: 0.0011 (0.0680) LR: 3.769e-04 Time: 0.783s,   10.21/s (0.966s,    8.28/s) Data: 0.585 (0.617)
/home/qjh

Compute PRO curve...
Sort 5439488 anomaly scores...


/home/qjh/MemSeg-main/metrics/pro_curve_util.py:104: RuntimeWarning: invalid value encountered in divide
  np.divide(pro_changes_sorted, num_gt_regions, out=pro_changes_sorted)
TEST: AUROC-image: 0.493% | Best-F1-score: 0.863% | Best-threshold: 0.508%


Best Score 0.000% to 62.138%


In [ ]:
from IPython.display import Image, display
display(Image(f"saved_model/MemSeg-{target}/{target}/test/good/000_heatmap.png"))
display(Image(f"saved_model/MemSeg-{target}/{target}/test/good/001_heatmap.png"))
display(Image(f"saved_model/MemSeg-{target}/{target}/test/good/004_heatmap.png"))
display(Image(f"saved_model/MemSeg-{target}/{target}/test/good/007_heatmap.png"))

In [ ]:
import mlflow
!mlflow ui --backend-store-uri file:'./mlruns'